# Load pytorch model

In [1]:
import torch
import numpy as np
import cv2
import os

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg

from detectron2.utils.logger import setup_logger
from detectron2.evaluation import inference_context
import detectron2.data.detection_utils as utils
from detectron2.utils import comm

# for inpainting
from deepfill.config import add_inpainter_config
from deepfill.train_loop import InpainterTrainer
import deepfill.dataset
import deepfill.inpnet
import deepfill.inpnet_tf

# tools for visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18.0, 18.0) # set default size of plots

import logging
from detectron2.utils.logger import setup_logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

# set configuration
config_file = "./configs/pretrained_eval_tf.yaml"
cfg = get_cfg()
add_inpainter_config(cfg)
cfg.merge_from_file(config_file)
cfg.freeze()
setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="deepfill")

# load model
pytorch_model = InpainterTrainer.build_model(cfg)
DetectionCheckpointer(pytorch_model, save_dir=cfg.OUTPUT_DIR).resume_or_load(cfg.MODEL.WEIGHTS, resume=False)

INFO:root:test
INFO:detectron2.engine.defaults:Model:
tf_GatedCNNSNPatchGAN(
  (inpaint_net): GatedCNN(
    (conv1): GatedConv2d(5, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv2_downsample): GatedConv2d(24, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): GatedConv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_downsample): GatedConv2d(48, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): GatedConv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv6): GatedConv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv7_atrous): GatedConv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (conv8_atrous): GatedConv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(4, 4), dilation=(4, 4))
    (conv9_atrous): GatedConv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(8, 8), dilation=(8, 8))
    (conv10_atrous): GatedConv2d(96, 192,

{}

# Load tensorflow model
modify the TensorFlow model to output the hidden-states at regular locations along the depth of the model

In [2]:
from tf_codes.inpaint_model import InpaintCAModel
import neuralgym as ng
import tensorflow as tf

FLAGS = ng.Config('inpaint.yml')
tf_model = InpaintCAModel()

/home/s2006466/anaconda3/envs/deepfill_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/s2006466/anaconda3/envs/deepfill_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/s2006466/anaconda3/envs/deepfill_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/s2006466/anaconda3/

---------------------------------- APP CONFIG ----------------------------------
num_gpus_per_job: 1
num_cpus_per_job: 4
num_hosts_per_job: 1
memory_per_job: 32
gpu_type: nvidia-tesla-p100
name: places2_gated_conv_v100
model_restore: 
dataset: celebahq
random_crop: False
val: False
log_dir: logs/full_model_celeba_hq_256
gan: sngan
gan_loss_alpha: 1
gan_with_mask: True
discounted_mask: True
random_seed: False
padding: SAME
train_spe: 4000
max_iters: 100000000
viz_max_out: 10
val_psteps: 2000
data_flist: 
  celebahq: ['data/celeba_hq/train_shuffled.flist', 'data/celeba_hq/validation_static_view.flist']
  celeba: ['data/celeba/train_shuffled.flist', 'data/celeba/validation_static_view.flist']
  places2: ['data/places2/train_shuffled.flist', 'data/places2/validation_static_view.flist']
  imagenet: ['data/imagenet/train_shuffled.flist', 'data/imagenet/validation_static_view.flist']
static_view_size: 30
img_shapes: [256, 256, 3]
height: 128
width: 128
max_delta_height: 32
max_delta_width: 32

# forward pytorch model and tensorflow model to compare the hidden outputs

In [3]:
layer_output_names = [
    "input",  # 
    "conv1",
    "conv2_downsample", "conv3",
    "conv4_downsample", "conv5", 'conv6',
    "conv7_atrous", "conv8_atrous", "conv9_atrous", "conv10_atrous",
    "conv11", "conv12",
    "conv13_upsample", "conv14",
    "conv15_upsample", "conv16", "conv17",
    "stage2_input",  # 
    "xconv1",
    "xconv2_downsample", "xconv3",
    "xconv4_downsample", "xconv5", "xconv6",
    "xconv7_atrous", "xconv8_atrous", "xconv9_atrous", "xconv10_atrous",
    "pmconv1",
    "pmconv2_downsample", "pmconv3",
    "pmconv4_downsample", "pmconv5", "pmconv6",
    "attention_output",  #
    "pmconv9", "pmconv10",
    "decoder_input",  # 
    "allconv11", "allconv12",
    "allconv13_upsample", "allconv14",
    "allconv15_upsample", "allconv16", "allconv17"
]
# layer_output_names = layer_output_names[:3]
# torch.backends.cudnn.deterministic = True
def forward_on_image(im_file_name, mask_file_name):
    # get the image and mask
    image = utils.read_image(im_file_name, format="BGR")  # this will be returned
    H, W = image.shape[:2]
    H = H // 8 * 8
    W = W // 8 * 8
    image = image[:H, :W, :]
    mask_im = cv2.imread(mask_file_name)  # this will be used to erase image that will be returned
    mask = (mask_im[:H, :W, 0] > 127.5)

    # input for pytorch model
    pytorch_model_input = {}
    pytorch_model_input["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
    pytorch_model_input["mask"] = torch.as_tensor(mask.astype(np.float32)[None])  # (1, H, W)
    pytorch_model_input["height"] = image.shape[0]
    pytorch_model_input["width"] = image.shape[1]
    
    # forward to get pytorch model hidden_output
    with inference_context(pytorch_model), torch.no_grad():
        pytorch_hidden_outputs = pytorch_model.get_hidden_outputs([pytorch_model_input])
    pytorch_hidden_outputs = [x.cpu().numpy() for x in pytorch_hidden_outputs]
        
    # input for tf model
    input_image = np.concatenate([image[None], mask_im[None]], axis=3)
    checkpoint_dir = "./output/pretrained/release_places2_256_deepfill_v2/"
    
    # forward to get tf model hidden_output
    tf.reset_default_graph()
    sess_config = tf.ConfigProto()
    sess_config.gpu_options.allow_growth = True
    with tf.Session(config=sess_config) as sess:
        input_image = tf.constant(input_image, dtype=tf.float32)
        outputs = tf_model.build_compare_graph(FLAGS, input_image)
        # load pretrained model
        vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        assign_ops = []
        for var in vars_list:
            vname = var.name
            from_name = vname
            var_value = tf.contrib.framework.load_variable(checkpoint_dir, from_name)
            assign_ops.append(tf.assign(var, var_value))
        sess.run(assign_ops)
        print('Model loaded.')
        tf_hidden_outputs = sess.run(outputs)
    tf_hidden_outputs = [x.transpose(0, 3, 1, 2) for x in tf_hidden_outputs]
    
    # compare the hidden_outputs
#     assert len(layer_output_names) == len(pytorch_hidden_outputs) == len(tf_hidden_outputs)
#     for name, pytorch_tensor, tf_tensor in zip(layer_output_names, pytorch_hidden_outputs, tf_hidden_outputs):
#         max_abs_val = np.abs(pytorch_tensor - tf_tensor).max()
#         if max_abs_val < 1e-4:
#             print(f"layer {name}: pass. with max abs val {max_abs_val}")
#         else:
#             print(f"the consistency check stopped at layer {name} with max abs val {max_abs_val}")
    return pytorch_hidden_outputs, tf_hidden_outputs
            
torch_ho, tf_ho = forward_on_image("./examples/places2/case1_input.png", "./examples/places2/case1_mask.png")

/home/s2006466/anaconda3/envs/deepfill_v2/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Model loaded.


In [4]:
for name, pytorch_tensor, tf_tensor in zip(layer_output_names, torch_ho, tf_ho):
    max_abs_val = np.abs(pytorch_tensor - tf_tensor).max()
    if max_abs_val < 1e-4:
        print(f"layer {name}: pass. with max abs val {max_abs_val}")
    else:
        print(f"the consistency check stopped at layer {name} with max abs val {max_abs_val}")

layer input: pass. with max abs val 1.1920928955078125e-07
layer conv1: pass. with max abs val 1.5497207641601562e-06
layer conv2_downsample: pass. with max abs val 2.1457672119140625e-06
layer conv3: pass. with max abs val 2.6226043701171875e-06
layer conv4_downsample: pass. with max abs val 3.0994415283203125e-06
layer conv5: pass. with max abs val 4.5299530029296875e-06
layer conv6: pass. with max abs val 1.0609626770019531e-05
layer conv7_atrous: pass. with max abs val 1.1444091796875e-05
layer conv8_atrous: pass. with max abs val 1.049041748046875e-05
layer conv9_atrous: pass. with max abs val 8.58306884765625e-06
layer conv10_atrous: pass. with max abs val 7.510185241699219e-06
layer conv11: pass. with max abs val 8.58306884765625e-06
layer conv12: pass. with max abs val 1.1920928955078125e-05
layer conv13_upsample: pass. with max abs val 1.0125339031219482e-05
layer conv14: pass. with max abs val 4.470348358154297e-06
layer conv15_upsample: pass. with max abs val 9.7751617431640

In [5]:
torch_w = torch_ho[-1].reshape((1, 96, 3, 3, 64, 85))
tf_w = tf_ho[-1].reshape((1, 3, 3, 96, 64, 85)).transpose((0, 3, 1, 2, 4, 5))
max_abs_val = np.abs(torch_w - tf_w).max()
print(max_abs_val)
# w = max_abs_val % 170
# h = max_abs_val // 170 % 128
# c = max_abs_val // 170 // 128
# print(c, h, w)

1.1321157e-05


In [6]:
torch_w

array([[[[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 2.10903734e-02, 9.08323303e-02, ...,
            1.77214652e-01, 1.41832441e-01, 1.66115507e-01],
           [0.00000000e+00, 8.43306854e-02, 1.99802294e-01, ...,
            9.71044600e-02, 7.56794959e-02, 1.61624119e-01],
           ...,
           [0.00000000e+00, 2.49511725e-03, 1.45407403e-02, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 9.50689763e-02, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 0.00000000e+00, 2.16443520e-02, ...,
            1.00612491e-02, 0.00000000e+00, 4.57912823e-03]],

          [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [2.10903734e-02, 9.08323303e-02, 1.05497845e-01, ...,
            1.41832441e-01, 1.66115

In [7]:
tf_w

array([[[[[[0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.02109034, 0.0908322 , ..., 0.17242514,
            0.14199872, 0.1204631 ],
           [0.        , 0.08433083, 0.19980262, ..., 0.08495723,
            0.09408389, 0.1637281 ],
           ...,
           [0.        , 0.05348091, 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.12635782, 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.06682002, 0.        , ..., 0.04011405,
            0.02162234, 0.        ]],

          [[0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.02109034, 0.0908322 , 0.10549781, ..., 0.14199872,
            0.1204631 , 0.12929097],
           [0.08433083, 0.19980262, 0.22148651, ..., 0.09408389,
            0.1637281 , 0.17234099],
           ...,
           [0.05348091, 0.        , 0.        , 

In [22]:
import torch.nn.functional as F

a = np.arange(16*16).reshape((16, 16))[None, None].astype(np.float32)
print(a)
torch_downsampling = F.interpolate(F.pad(torch.from_numpy(a), (0, 1, 0, 1)), size=(8,8) ) #, scale_factor=0.5)
print(torch_downsampling)

[[[[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.
     13.  14.  15.]
   [ 16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
     29.  30.  31.]
   [ 32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
     45.  46.  47.]
   [ 48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.
     61.  62.  63.]
   [ 64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.
     77.  78.  79.]
   [ 80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.
     93.  94.  95.]
   [ 96.  97.  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108.
    109. 110. 111.]
   [112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124.
    125. 126. 127.]
   [128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140.
    141. 142. 143.]
   [144. 145. 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156.
    157. 158. 159.]
   [160. 161. 162. 163. 164. 165. 166. 167. 168. 169. 170. 171. 172.
    173. 174. 175.]
   [176. 177. 178. 17

In [20]:
from neuralgym.ops.layers import resize

with tf.Session() as sess:
    input_image = tf.constant(a[0, :, :, :, None], dtype=tf.float32)
    outputs = resize(input_image, scale=0.5, func=tf.image.resize_nearest_neighbor)
    tf_downsampling = sess.run(outputs)
print(tf_downsampling.transpose(0,3, 1, 2))

[[[[  0.   2.   4.   6.   9.  11.  13.  15.]
   [ 32.  34.  36.  38.  41.  43.  45.  47.]
   [ 64.  66.  68.  70.  73.  75.  77.  79.]
   [ 96.  98. 100. 102. 105. 107. 109. 111.]
   [144. 146. 148. 150. 153. 155. 157. 159.]
   [176. 178. 180. 182. 185. 187. 189. 191.]
   [208. 210. 212. 214. 217. 219. 221. 223.]
   [240. 242. 244. 246. 249. 251. 253. 255.]]]]


In [27]:
np.linspace(0, 15, 8).ndim

1

In [29]:
inds = torch.linspace(0, 15, 8).sub(0.5).ceil().long()
y_inds, x_inds = torch.meshgrid(inds, inds)

In [31]:
x_inds

tensor([[ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15],
        [ 0,  2,  4,  6,  9, 11, 13, 15]])

In [5]:
inds = torch.linspace(0, 15, 8).add(0.5).floor().long()

In [6]:
inds

tensor([ 0,  2,  4,  6,  9, 11, 13, 15])

In [7]:
torch.linspace(0, 15, 8).add(0.5).floor()

tensor([ 0.,  2.,  4.,  6.,  9., 11., 13., 15.])